In [ ]:
'ADC to FFT empty seat new approach'


# Loading the Data

In [ ]:
import gc
gc.collect()
label_value = 1 
label_column_name = 'Object_Presence'

In [ ]:
%reset -f

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

In [ ]:
# Set directory containing CSV files
data_dir = r"/Users/mandarkale/Documents/MyProjects/MachineLearning/Raw/Emptyseat/"

# Load all CSV files
csv_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.csv')]
dfs = [pd.read_csv(file, header=None) for file in csv_files]


In [ ]:
# Combine all CSV data
combined_df = pd.concat(dfs, ignore_index=True)
combined_df

In [ ]:
combined_df.shape

In [ ]:
adc_data = combined_df.iloc[:, 16:].values  # Extract ADC values (skip metadata columns)
adc_data
#print(f"Loaded data shape: {adc_data.shape}")  # Shape should be (num_samples, num_features)


In [ ]:
from scipy.signal import butter, filtfilt

def butter_lowpass_filter(signal, cutoff=30e3, fs=1e6, order=5):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return filtfilt(b, a, signal)

filtered_data = np.array([butter_lowpass_filter(row) for row in adc_data])
filtered_data

In [ ]:
def calculate_snr(signal, noise):
    power_signal = np.mean(signal ** 2)
    power_noise = np.mean(noise ** 2)
    return 10 * np.log10(power_signal / power_noise)

snr_after = calculate_snr(filtered_data, adc_data - filtered_data)
print(f"SNR After Fixing: {snr_after:.2f} dB")

In [ ]:
# Select from the 17th column to the end
adc_data_selected_columns = filtered_data.mean(axis=1)  # Python uses 0-based indexing
adc_data_selected_columns

# ADC to FFT Conversion

In [ ]:
# Convert the pandas Series to a numpy array
adc_array = adc_data_selected_columns 

# Choose a window function - Hanning window in this case
window = np.hanning(len(adc_array))

# Apply the window function to your data
windowed_adc_data = adc_array * window

# Perform FFT on the windowed data
fft_result = np.fft.fft(windowed_adc_data)

# Frequency bins (assuming you know the sampling rate)
sampling_rate = 1953125  # Example: 1000 Hz, replace with your actual sampling rate
min_freq = 35000  # 35 kHz
max_freq = 45000  # 45 kHz

n = len(adc_array)
freq = np.fft.fftfreq(n, d=1/sampling_rate)
# Calculate the magnitude and phase of the FFT result
magnitude = np.abs(fft_result)
phase = np.angle(fft_result)

# Create a DataFrame
fft_df = pd.DataFrame({
    'Frequency': freq,
    'FFT Magnitude': magnitude,
    'Phase': phase
})

fft_df.head()  # Display the first few rows of the DataFrame

In [ ]:
# Add a new column to the fft_magnitude DataFrame for the binary label
# Set the value to 1 for presence of an infant with a carriage
#fft_df['Object_Presence'] = 0  # 1 for presence, 0 for absence (if you have such data)
fft_df[label_column_name] = label_value

# Display the first few rows to verify the new column
fft_df.head()

In [ ]:
len(fft_df)

In [ ]:
fft_df

In [ ]:
plt.figure(figsize=(10, 6))  # Set the figure size for better readability
plt.plot(fft_df['Frequency'], fft_df['FFT Magnitude'])  # Plot positive frequency vs magnitude
plt.title('Magnitude Spectrum')  # Title of the plot
plt.xlabel('Frequency (Hz)')  # Label for the x-axis
plt.ylabel('Magnitude')  # Label for the y-axis
plt.grid(True)  # Show grid for better readability
# Save the figure as PNG
fft_plot_path = os.path.join(
    "/Users/mandarkale/Documents/MyProjects/MachineLearning/FIUS_Based_Infant_Presence_Detection_in_Car_Seats/source_code/Data/Images",
    "FFT_EmptySeat.png"
)
plt.savefig(fft_plot_path)

# Optionally close the plot to free memory
plt.close()
plt.show()  # Display the plot



# Saving to numpy array

In [ ]:
numpy_array = fft_df.to_numpy()
# Save the array to a file
#np.save('source_code/Data/Processed/Emptyseat_npy_array_Lowpassfiltered_label.npy', numpy_array)

# Define save path (2 levels up → Data/Processed/)
# save_path = os.path.join("..", "..", "Data", "Processed", "Emptyseat_npy_array_Lowpassfiltered_label.npy")

project_root = "/Users/mandarkale/Documents/MyProjects/MachineLearning/FIUS_Based_Infant_Presence_Detection_in_Car_Seats/source_code"
save_path = os.path.join(project_root, "Data", "Processed", "Emptyseat_npy_array_Lowpassfiltered_label.npy")



# If file already exists, remove it
if os.path.exists(save_path):
    os.remove(save_path)
    print(f"Old file removed: {save_path}")

# Save the array to that location
np.save(save_path, numpy_array)

print(f"New file saved at: {os.path.abspath(save_path)}")

In [ ]:
import gc
gc.collect()

In [ ]:
%reset -f